# Alternative Methods

Due to the relationship between the Chebyshev and Fourier bases, there are two major ways to take a derivative in the Chebyshev basis:

1. Via Fourier basis  
   a. Sample the function at Chebyshev points and then pretend you're actually equisampling in a different domain, $\theta$ instead of $x$.  
   b. Transfrom via DCT to get to frequency domain $\omega$.  
   c. Multiply by appropriate $jk$ as when taking Fourier-spectral derivatives.  
   d. Transform back to $\theta$ domain with DCT or DST, depending on evenness or oddness of derivative order.  
   e. Transform back to the $x$ domain via complicated mapping, derived with chain rule.  
   f. Treat the endpoints carefully, because the transform from $\theta$ to $x$ can't be solved explicitly at $\pm 1$.  
2. Via [series coefficient rule](https://scicomp.stackexchange.com/questions/44939/chebyshev-series-derivative-in-terms-of-coefficients)  
   a. Also use Chebyshev points so we can transform from function samples to Chebyshev coefficients in $O(N \log N)$ (although you could technically use any points you like if you're willing to fit the Chebyshev polynomials in $O(N^2)$).  
   d. DCT and do some additional O(N) divisions to get said coefficients.  
   c. Use the derivative rule to get Chebyshev coefficients of the derivative function in Chebyshev series representation.  
   d. Do some additional O(N) multiplications to make DCT coefficients from Chebyshev coefficients and DCT to get function samples in $O(N \log N)$ (or evaluate the function at any points you like in $O(N^2)$).  

In [On the Errors Incurred Calculating Derivatives Using Chebyshev Polynomials](https://www.sciencedirect.com/science/article/pii/0021999192902743) the athors first go through method 2 but find that "Tight coupling between coefficients enaboes propagation of errors from high frequency to low frequency modes." They then explore method 1, which up until the mapping from $\theta$ back to $x$ has cleaner separation of modes as in the Fourier case. However, [division by powers of $\sqrt{1-x^2}$](https://pavelkomarov.com/spectral-derivatives/math.pdf) then blows up any noise or inexactness in the representation of the function at the edges of the domain ($x \rightarrow \pm 1$). They suggest a third method, which I've found to be no better, covered below.

In [1]:
import numpy as np
from scipy.fft import dct, dst
from numpy.polynomial import Chebyshev, Polynomial as poly
from collections import deque
from warnings import warn

## Implementation of Method 1

This was the original method in `spec_deriv.py`, because it's essentially the method suggested by [Trefethen](https://epubs.siam.org/doi/epdf/10.1137/1.9780898719598.ch8). I've extended this method significantly in [my own analysis](https://pavelkomarov.com/spectral-derivatives/math.pdf) and spill quite a bit of ink over it, which I've decided to keep for sake of completeness and intuition. Best to keep the code for reference and comparison to other methods.

In [2]:
def cheb_deriv_via_fourier(y_n: np.ndarray, t_n: np.ndarray, order: int, axis: int=0, filter: callable=None, dct_type=1, calc_endpoints=True):
	"""Evaluate derivatives with Chebyshev polynomials via discrete cosine and sine transforms. Caveats:

	- Taking the 1st derivative twice with a discrete method like this is not exactly the same as taking the second derivative.
	- For derivatives over the 4th, this method presently returns :code:`NaN` at the edges of the domain. Be cautious if passing
	  the result to another function.

	Args:
		y_n (np.ndarray): one-or-multi-dimensional array, values of a function, sampled at cosine-spaced points in the dimension
			of differentiation.
		t_n (np.ndarray): 1D array, where the function :math:`y` is sampled in the dimension of differentation. If you're using
			canonical Chebyshev points with the DCT-I, this will be :code:`x_n = np.cos(np.arange(N+1) * np.pi / N)` (:math:`x
			\\in [1, -1]`). With the DCT-II, :code:`x_n = np.concatenate(([1], np.cos((np.arange(N+1) + 0.5) * np.pi/(N+1)), [-1]))`.
			In either case, on a domain from :math:`a` to :math:`b`, this is stretched to :code:`t_n = x_n * (b - a)/2 + (b + a)/2`.
			Note the order is high-to-low in the :math:`x` or :math:`t` domain, but low-to-high in :math:`n`. Also note both
			endpoints are *inclusive*.
		order (int): The order of differentiation, also called :math:`\\nu`. Must be :math:`\\geq 1`.
		axis (int, optional): For multi-dimensional :code:`y_n`, the dimension along which to take the derivative. Defaults to the
			first dimension (axis=0).
		filter (callable, optional): A function or :code:`lambda` that takes the 1D array of Chebyshev polynomial numbers, :math:`k
			= [0, ... N]`, and returns a same-shaped array of weights, which get multiplied in to the initial frequency transform of
			the data, :math:`Y_k`. Can be helpful when taking derivatives of noisy data. The default is to apply #nofilter.
		dct_type (int, optional): 1 or 2, whether to use DCT-I or DCT-II. Defaults to DCT-I.
		calc_endpoints (bool, optional): Whether to calculate the endpoints of the answer, in case they are unnecessary for a
			particular use case. Defaults to :code:`True`. :code:`False` silences the :code:`NaN` warning for :code:`order` :math:`> 4`.
 
	:returns: (*np.ndarray*) -- :code:`dy_n`, shaped like :code:`y_n`, samples of the :math:`\\nu^{th}` derivative of the function
	"""
	# We only have to care about the number of points in the dimension we're differentiating
	N = y_n.shape[axis] - 1 if dct_type == 1 else y_n.shape[axis] - 3 # if type is 1, we count [0, ... N], if type 2, the endpoints are tacked on additionally
	M = 2*N if dct_type == 1 else 2*(N+1) # Normalization factor is larger for DCT-II based on repeats of endpoints in equivalent FFT
	x_n = np.cos(np.arange(N+1) * np.pi/N) if dct_type == 1 else np.concatenate(([1], np.cos((np.arange(N+1) + 0.5) * np.pi/(N+1)), [-1])) # canonical sampling

	if order < 1:
		raise ValueError("derivative order, nu, should be >= 1")
	if dct_type not in (1, 2):
		raise ValueError("DCT type must be 1 or 2")
	if len(t_n.shape) > 1 or t_n.shape[0] != y_n.shape[axis]:
		raise ValueError("t_n should be 1D and have the same length as y_n along the axis of differentiation")
	if not np.all(np.diff(t_n) < 0):
		raise ValueError("The domain, t_n, should be ordered high-to-low, [b, ... a]. Try sampling with `np.cos(np.arange(N+1) * np.pi / N) * (b - a)/2 + (b + a)/2`")
	scale = (t_n[0] - t_n[-1])/2; offset = (t_n[0] + t_n[-1])/2 # Trying to be helpful, because sampling is tricky to get right
	if not np.allclose(t_n, x_n * scale + offset, atol=1e-5):
		raise ValueError(f"""Your function is not sampled appropriately for the DCT-{'I'*dct_type}. Try sampling with
			{'`np.cos(np.arange(N+1) * np.pi / N) * (b - a)/2 + (b + a)/2`' if dct_type == 1 else
			'`np.concatenate(([b], np.cos((np.arange(N+1) + 0.5) * np.pi/(N+1)) * (b - a)/2 + (b + a)/2, [a]))`'}""")

	first = [slice(None) for dim in y_n.shape]; first[axis] = 0; first = tuple(first) # for accessing different parts of data
	last = [slice(None) for dim in y_n.shape]; last[axis] = -1; last = tuple(last)
	middle = [slice(None) for dim in y_n.shape]; middle[axis] = slice(1, -1); middle = tuple(middle)
	s = [np.newaxis for dim in y_n.shape]; s[axis] = slice(None); s = tuple(s) # for elevating vectors to have same dimension as data

	Y_k = dct(y_n, 1, axis=axis) if dct_type == 1 else dct(y_n[middle], 2, axis=axis) # Transform to frequency domain using the 1st definition of the discrete cosine transform
	k = np.arange(N+1) # [0, ... N], Chebyshev basis polynomial (in x)/wavenumber (in theta) iterator
	if filter: Y_k *= filter(k)[s]

	y_primes = [] # Store all derivatives in theta up to the nu^th, because we need them all for reconstruction.
	for mu in range(1, order + 1):
		Y_mu = (1j * k[s])**mu * Y_k
		if mu % 2: # odd derivative
			# In DST-I case Y_mu[k=0 and N] = 0 and so are not needed for the DST, so only pass the [middle] entries
			# In DST-III case, Y_mu[0 and N+1] = 0. roll() shifts to the left, so Y'_0 is treated like Y'_{N+1}, and we pass in starting at k=1
			y_primes.append(dst(1j * Y_mu[middle], 1, axis=axis).real / M if dct_type == 1 # d/dtheta y = the inverse transform of DST-1 = 1/M * DST-1. Extra j for equivalence with IFFT.
				else dst(1j * np.roll(Y_mu, -1), 3, axis=axis).real / M) # inverse of DST-II is 1/M * DST-III. Im{y_prime} = 0 for real y, so just keep real.
		else: # even derivative
			y_primes.append(dct(Y_mu, 1, axis=axis)[middle].real / M if dct_type == 1 # the inverse transform of DCT-1 is 1/M * DCT-1. Slice off ends to get same length as DST-I result.
				else dct(Y_mu, 3, axis=axis).real / M) # inverse of DCT-II is 1/M * DCT-III. Im{y_prime} = 0 for real y, so just keep real.

	# Calculate the polynomials in x necessary for transforming back to the Chebyshev domain
	numers = deque([poly([-1])]) # just -1 to start, at order 1
	denom = poly([1, 0, -1]) # 1 - x^2
	for nu in range(2, order + 1): # initialization takes care of order 1, so iterate from order 2
		q = 0
		for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
			p = numers.popleft() # c = nu - mu/2
			numers.append(denom * p.deriv() + (nu - mu/2 - 1) * poly([0, 2]) * p - q)
			q = p
		numers.append(-q)
	
	# Calculate x derivative as a sum of x polynomials * theta-domain derivatives
	dy_n = np.zeros(y_n.shape) # The middle of dy will get filled with a derivative expression in terms of y_primes
	denom_x = denom(x_n[1:-1]) # only calculate this once; leave off +/-1, because they need to be treated specially anyway
	for term,(numer,y_prime) in enumerate(zip(numers, y_primes), 1): # iterating from lower derivatives to higher
		c = order - term/2 # c starts at nu - 1/2 and then loses 1/2 for each subsequent term
		dy_n[middle] += (numer(x_n[1:-1])/np.power(denom_x, c))[s] * y_prime

	# Calculate the endpoints
	alt = np.ones(N+1); alt[1::2] = -1 # alternating for summing the last point
	if order <= 4 and calc_endpoints:
		C, D = {1: [(-1,), 1], 2: [(1, 1), 3], 3: [(-4, -5, -1), 15], 4: [(36, 49, 14, 1), 105]}[order] # Constants from the math. See the notebook in the warning.
		LH = 0 # L'Hôpital numerator terms
		for i,C_i in enumerate(C, 1): # i starts at 1
			LH += 2 * C_i * (-1)**i * np.power(k, 2*i)
			if dct_type == 1: LH[-1] -= C_i * (-1)**i * np.power(N, 2*i) # because Nth element is outside the 2\sum in the DCT-I
		dy_n[first] = np.sum(LH[s] * Y_k, axis=axis)/ (D*M)
		dy_n[last] = np.sum((LH * alt)[s] * Y_k, axis=axis) / ((-1)**order * D*M)
	else: # For higher derivatives, leave the endpoints uncalculated, but direct the user to my analysis of this problem.
		if calc_endpoints: warn("""endpoints set to NaN, only calculated for 4th derivatives and below. For help with higher derivatives,
			see https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/chebyshev_domain_endpoints.ipynb""")
		dy_n[first] = dy_n[last] = np.nan

	# The above is agnostic to where the data came from, pretends it came from the domain [-1, 1], but the data may actually be
	return dy_n/scale**order # smooshed from some other domain. So scale the derivative by the relative size of the t and x intervals.


## Method 2

For an explanation see the [series-to-series differentation](https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/chebyshev_series_based_derivatives.ipynb) notebook. For a complete implementation of this method, see [`spec_deriv.py`](https://github.com/pavelkomarov/spectral-derivatives/blob/main/specderiv/specderiv.py). I've chosen this method to live in the library code, because it is the simplest and shortest to read, write, and understand, comes with other benefits like allowing arbitrary $x$ points (if you're willing to suffer $O(N^2)$ work), and it works essentially numerically identically to Method 1, [even in the presence of noise](https://github.com/pavelkomarov/spectral-derivatives/blob/main/notebooks/filtering_noise.ipynb), which is maybe surprising.

## Method 3

The authors of [On the Errors Incurred Calculating Derivatives Using Chebyshev Polynomials](https://www.sciencedirect.com/science/article/pii/0021999192902743) suggest writing the function to be differentiated, which I'll call $y$ for consistency with myself, as:

$$y(x) = \frac{1+x}{2}y(1) + \frac{1-x}{2}y(-1) + (1-x^2)g(x)$$

Notice that as $x \rightarrow 1$ the rightmost two terms disappear, and we're left with $y(1)$, and similarly for the other end of the domain, so this is a valid representation. We can now take a derivative to find:

$$y'(x) = \frac{y(1) - y(-1)}{2} + \underbrace{(1-x^2)}_{w(x)}g'(x) + \underbrace{-2x}_{w'(x)} g(x)$$

Their claim is that the errors at the edges of the domain accrued in the differentiation process to find $g'$ are now somewhat damped by the weighting term $1-x^2$. However, notice that if all we have to start with is $y$, and $y$ is *numerical* rather than analytic, containing some kind of noise, then when we find $g$ by algebraically manipulating the first equation, we'll end up having to divide by $1-x^2$, which will blow up noise from the edges of $y$. Not great.

If we substitute $g$ in terms of $y$ back in to $y'$ we get:

$$y'(x) = \frac{y(1) - y(-1)}{2} + w(x)g'(x) + \frac{w'(x)}{w(x)} \Big( \frac{y(x) - \frac{1+x}{2}y(1) - \frac{1-x}{2}y(-1)}{1-x^2} \Big)$$

So really in order to shrink any blowup in both $g'$ and $g$, we need both $w$ and $\frac{w'}{w}$ to go to zero at the domain edges. This turns out to be impossible. Proof:

$$ \frac{w'(x)}{w(x)} = \frac{d}{dx} \ln(w(x)) =: h'(x) $$
$$ w(x) \rightarrow 0 \iff h(x) = \ln(w(x)) \rightarrow -\infty$$

So we need $h$ to go to $-\infty$ while $h'$ goes to 0, i.e. approach negative infinity with zero slope. This is possible on an infinite interval, but we are on the finite interval $[-1, 1]$, so no such $h$ can exist and therefore no such $w$. $\blacksquare$

This fundamental limitation is unfortunate but not completely unexpected, since uniform sample uncertainty $\Delta y$ over closer and closer samples $\Delta x \rightarrow \epsilon $ will fundamentally cause a blowup in $\frac{\Delta y}{\Delta x}$.

There is also an additional wrinkle caused by $w$ going to 0 at the domain edges, which makes it impossible to evaluate $g$ directly there. The authors reassure us that if $w$ is polynomial we can use orthogonality of Chebyshev polynomials to set up a system of equations to solve for the endpoints, but this is annoying.

However, since they *do not* touch on intentionally *filtering* noise by culling high-frequency modes, I wondered whether this method could indeed improve upon the others and coded it up.

In [3]:
def brown_deriv(y_n: np.ndarray, filter: callable=None, filter_input=False, filter_g=False):
	"""Brown paper method. We write y(x) = (1+x)y(1)/2 + (1-x)y(-1)/2 + (1-x^2)g(x), where we don't know g but can find g.
	Now we can take the derivative y'(x) = (y(1) - y(-1))/2 + (1-x^2)g'(x) - 2xg(x). We can get g' from g via the Chebyshev
	coefficients recursion. Both g and g' blow up at the edges, but the effect is balanced in the case of g' by multiplication
	with 1-x^2.
	
	Args:
		y_n (np.ndarray): one-or-multi-dimensional array, values of a function, sampled at cosine-spaced points in the dimension
			of differentiation.
		filter (callable, optional): A function or :code:`lambda` that takes the 1D array of Chebyshev polynomial numbers, :math:`k
			= [0, ... N]`, and returns a same-shaped array of weights, which get multiplied in to the initial frequency transform of
			the data, :math:`Y_k`. Can be helpful when taking derivatives of noisy data. The default is to apply #nofilter.
		filter_input (bool, optional): Whether to apply the filter to :math:`y` at input-time.
		filter_g (bool, optional): Whether to apply the filter to :math:`g`, deeper in the process.

	:returns: (*np.ndarray*) -- :code:`dy_n`, shaped like :code:`y_n`, samples of the :math:`\\nu^{th}` derivative of the function
	"""
	N = len(y_n) - 1
	if filter_input:
		Y_k = dct(y_n, 1)
		y_n = dct(Y_k * filter(np.arange(N+1)), 1)/(2*N)

	# Find g
	g_n_interior = (y_n[1:-1] - (1+x_n[1:-1])/2 * y_n[0] - (1-x_n[1:-1])/2 * y_n[-1])/(1-x_n[1:-1]**2) # y_n is ordered from 1 to -1
	# Find the endpoints of g using auxilliary orthogonality equations: sum(g(x_n)*T_k(x_n)) = 0 for k > N-2, because we know g is
	# made of polynomials of order <= N-2, therefore its inner product with T_{N-1} and T_N should be 0. Two equations for two
	# unknowns. T_N evaluated at the Chebyshev points is just [1, -1, 1, -1 ...], which we can create in O(N), and we can evaluate
	# any other with a DCT in O(N log N): Chebyshev([0, 0, ... 0, 1, 0])(x_n) == dct([0, 0, ... 0, 1, 0], 1)/2 <-- note the ordering from 1 to -1
	T_N = np.ones(N+1); T_N[1::2] = -1 # ordered from 1 to -1
	coefsNm1 = np.zeros(N+1); coefsNm1[N-1] = 1
	T_Nm1 = dct(coefsNm1, 1)/2 # There's a 2 outside the DCT-I sum, which will multiply the N-1th coefficient in the Chebyshev sum; cancel it by dividing
	b1 = -T_N[1:-1] @ g_n_interior
	b2 = -T_Nm1[1:-1] @ g_n_interior
	# There is an alternative way to find the endpoints if you know the value of y' at the endpoints, which we know how to find
	# from handling the last step of Method 1:
	# Y_k = dct(y_n, 1)
	# k = np.arange(1, N)
	# y_prime_1 = np.sum(k**2 * Y_k[1:-1])/N + (N/2) * Y_k[-1]
	# y_prime_m1 = -np.sum((k**2 * np.power(-1, k)) * Y_k[1:-1])/N - (N/2)*(-1)**N * Y_k[-1]
	# g_n[0] = 0.5*(0.5*(y_n[0] - y_n[-1]) - y_prime_1)
	# g_n[-1] = -0.5*(0.5*(y_n[0] - y_n[-1]) - y_prime_m1)
	# The above yields the solution to the following:
	# +g1 - 2b1 +/- gm1 = 0
	# +g1 - 2b2 -/+ gm1 = 0
	# which is actually treating the two endpoints as if their values are double the magnitude of the central points when we do
	# the inner product. Not sure why this is necessary, but without this the endpoints definitely don't align with the rest of g.
    # g1 always gets a + in the system of equations because T_k(1) = 1 for all k, and gm1 gets a +/- depending on odd/evenness of N.
    # This has a closed form solution:
	g1 = b1 + b2
	gm1 = b1 - b2 if N % 2 == 0 else -b1 + b2
	g_n = np.concatenate(([g1], g_n_interior, [gm1]))

	# DCT to get Chebyshev coefficients of g
	G_k = dct(g_n, 1)
	if filter_g:
		G_k *= filter(np.arange(N+1))
		g_n = dct(G_k, 1)/(2*N)
	# In the IDCT-I we have y_n = 1/(2N) (Y_0 + (-1)^n Y_N + 2 \sum_{k=1}^{N-1} cos(pi k n/N) Y_k), but in the Chebyshev
	# expansion we have: y_n = \sum_{k=0}^{N} cos(pi k n/N) a_k. So technically we need to do some scaling to get the
	# coefficients we actually need. But we can skip the uniform scaling by 1/N, because we'd just have to undo it later.  
	for k in (0, N): G_k[k] /= 2 # turn the DCT coefficients into scaled Chebyshev coefficients

	# Form the scaled Chebyshev polynomial and apply the rule to get the series of its derivative	
	cheb = np.polynomial.Chebyshev(G_k)
	dcheb = cheb.deriv()

	# IDCT to get values of g' back
	G_k_prime = np.concatenate((dcheb.coef, [0])) # Had we scaled by 1/N above, we'd have to multiply dcheb.coef by N here
	G_k_prime[0] *= 2 # We should technically also scale G_k_prime[N] by 2, but this entry is 0
	g_n_prime = dct(G_k_prime, 1)/(2*N)

	# Put together into y' and return
	return (y_n[0] - y_n[-1])/2 + (1-x_n**2)*g_n_prime - 2*x_n*g_n


## Comparison

I want to show these methods are all the same.

In [ ]:
N = 50
cutoff = 7
noise_scale = 0.1
x_n = np.cos(np.arange(N+1) * np.pi / N)
x = np.linspace(-1, 1, 1000, endpoint=True)

u = lambda x: np.exp(x)*np.sin(5*x)
du = lambda x: 5*np.exp(x) * np.cos(5*x) + np.exp(x) * np.sin(5*x)
#u = lambda x: np.sin(8*(x+1))/((x + 1.1)**(3/2))
y = lambda x: u(x) + np.random.randn(len(x))*noise_scale # y is u with noise




pyplot.plot(x, u(x), label='u')
pyplot.plot(x_n, u_n, 'C0+')
pyplot.plot(x[1:-1], (u(x)[1:-1] - (1+x[1:-1])/2 * u(x)[-1] - (1-x[1:-1])/2 * u(x)[0])/(1-x[1:-1]**2), label='g')
pyplot.plot(x_n, g_n, 'C1+')
#pyplot.plot(x_n, g_n_filtered, 'C3+')
pyplot.plot(x, du(x), 'C2', label='du')
pyplot.plot(x_n, u_n_prime, 'C2+')
pyplot.plot(x_n, cheb_deriv(u_n, x_n, 1, filter=filter), 'C2.')

pyplot.legend()
pyplot.show()


    	


#coefsN = np.array([0]*(N+1)); coefsN[N] = 1
coefsNm1 = np.zeros(N+1); coefsNm1[N-1] = 1

#chebN = np.polynomial.Chebyshev(coefsN)
chebNm1 = np.polynomial.Chebyshev(coefsNm1)

#print(chebN(x_n)) # O(N^2) to evaluate
# print(dct(coefsN,1)) # O(N log N) to evaluate. This is always 1, -1, 1, -1, ...
# e = np.ones(N+1); e[1::2] = -1 # O(N)
# print(e)

via_cheb = chebNm1(x_n)
via_dct = dct(coefsNm1, 1)/2
#print(via_cheb)
#print(via_dct)
assert np.allclose(via_dct, via_cheb)


# u_n = u(x_n)
# U_k = dct(u_n, 1)
# u_n_reconstruct = dct(U_k, 1)/(2*N)
# pyplot.plot(x, u(x))
# pyplot.plot(x_n, u_n, 'C0+')
# pyplot.plot(x_n, u_n_reconstruct, 'C1+')
# pyplot.legend()
# pyplot.show()

d(u(x_n), filter=None)#lambda k: np.abs(k) < cutoff)

